# Qrious Technical Development Task

## Pokedex using PokeAPI:

In [68]:
import sys

import numpy as np
import pandas as pd
import requests
import tkinter as tk
from tkinter import messagebox

In [58]:
response = requests.get('https://pokeapi.co/api/v2/pokemon/ditto')

# Checking if connection successful or not. 200 means successful. 
print(response.status_code)

# want to print data received from APi:
response_json = response.json()

#print(response_json)

# Extracting data from json
print(response_json['name'])
print(response_json['weight'])
print(response_json['height'])
print(response_json['base_experience'])

# Extracting data from nested json
print(response_json['abilities'])
print(response_json['abilities'][0]['ability']['name'])
print(response_json['abilities'][1]['ability']['name'])

# Extract data from first 151 pokemon:

# Initialize emoty array to store pokemon names:
pokemon_names = []

number_pokemon_names = 151

response = requests.get(f'https://pokeapi.co/api/v2/pokemon/{2}')
name = response.json()['name']

print("response is: ", response)
print("name is: ", name)



# Extract all pokemon names and append to pokemon_names array:
for i in range(1, 152):
    response = requests.get(f'https://pokeapi.co/api/v2/pokemon/{i}')
    response_json = response.json()
    pokemon_names.append(response_json['name'])
 
#check for loop accuracy: 
print("2nd pokemon name from loop: ", pokemon_names[1]) # should print ivysaur  



200
ditto
40
3
101
[{'ability': {'name': 'limber', 'url': 'https://pokeapi.co/api/v2/ability/7/'}, 'is_hidden': False, 'slot': 1}, {'ability': {'name': 'imposter', 'url': 'https://pokeapi.co/api/v2/ability/150/'}, 'is_hidden': True, 'slot': 3}]
limber
imposter
response is:  <Response [200]>
name is:  ivysaur
2nd pokemon name from loop:  ivysaur


In [59]:
# Extract all pokemon Number (id column in JSON) and append to pokemon_numbers array:
pokemon_numbers = []
for i in range(1, 152):
    response = requests.get(f'https://pokeapi.co/api/v2/pokemon/{i}')
    response_json = response.json()
    pokemon_numbers.append(response_json['id'])
    
#check for loop accuracy:
print("2nd pokemon number from loop: ", pokemon_numbers[1]) # should print 2

response = requests.get(f'https://pokeapi.co/api/v2/pokemon/{2}')
number = response.json()['id']

print("number is: ", number) # should print 2

2nd pokemon number from loop:  2
number is:  2


In [60]:
# Extract all pokemon weights and append to pokemon_weights array:
pokemon_weights = []
for i in range(1, 152):
    response = requests.get(f'https://pokeapi.co/api/v2/pokemon/{i}')
    response_json = response.json()
    pokemon_weights.append(response_json['weight'])

#check for loop accuracy:
print("2nd pokemon weight from loop: ", pokemon_weights[119]) # should print 345

response = requests.get(f'https://pokeapi.co/api/v2/pokemon/{120}')
weight = response.json()['weight']
print("weight is: ", weight) # should print 345

2nd pokemon weight from loop:  345
weight is:  345


In [61]:
# Extract all pokemon heights and append to pokemon_heights array:
pokemon_heights = []
for i in range(1, 152):
    response = requests.get(f'https://pokeapi.co/api/v2/pokemon/{i}')
    response_json = response.json()
    pokemon_heights.append(response_json['height'])

#check for loop accuracy:
print("2nd pokemon height from loop: ", pokemon_heights[90]) # should print 15

response = requests.get(f'https://pokeapi.co/api/v2/pokemon/{91}')
height = response.json()['height']
print("height is: ", height) # should print 15

2nd pokemon height from loop:  15
height is:  15


In [62]:
# Extract all pokemon base experiences and append to pokemon_base_experiences array:
pokemon_base_experiences = []
for i in range(1, 152):
    response = requests.get(f'https://pokeapi.co/api/v2/pokemon/{i}')
    response_json = response.json()
    pokemon_base_experiences.append(response_json['base_experience'])
    
#check for loop accuracy:
print("2nd pokemon base experience from loop: ", pokemon_base_experiences[90]) # should print 184

response = requests.get(f'https://pokeapi.co/api/v2/pokemon/{91}')
base_experience = response.json()['base_experience']
print("base experience is: ", base_experience) # should print 184

2nd pokemon base experience from loop:  184
base experience is:  184


In [63]:
# Extract all pokemon abilities and append to pokemon_abilities array:
pokemon_abilities = []
for i in range(1, 152):
    response = requests.get(f'https://pokeapi.co/api/v2/pokemon/{i}')
    response_json = response.json()
    pokemon_abilities.append(response_json['abilities'][0]['ability']['name'])

#check for loop accuracy:
print("2nd pokemon ability from loop: ", pokemon_abilities[88]) # should print stench

response = requests.get(f'https://pokeapi.co/api/v2/pokemon/{89}')
ability = response.json()['abilities'][0]['ability']['name']
print("ability is: ", ability) # should print stench


2nd pokemon ability from loop:  stench
ability is:  stench


In [64]:
# create new feature of string w combined position name and number: 
pokemon_names_numbers = []

for i in range(0, 151):
    pokemon_names_numbers.append(pokemon_names[i] + " " + str(pokemon_numbers[i]))

#check for loop accuracy:
print("2nd pokemon name and number from loop: ", pokemon_names_numbers[1]) # should print ivysaur 2
print(type(pokemon_names_numbers[1])) # should print <class 'str'>

2nd pokemon name and number from loop:  ivysaur 2
<class 'str'>


In [65]:
# organise extracted data to a pandas dataframe:
pokemon_df = pd.DataFrame({
    'Number': pokemon_numbers,
    'Name': pokemon_names,
    'Weight': pokemon_weights,
    'Height': pokemon_heights,
    'Base Experience': pokemon_base_experiences,
    'Ability': pokemon_abilities,
    'NameNumber' : pokemon_names_numbers
})

# reorganise data so can access pokemon details by name:
pokemon_df_name = pokemon_df.set_index('NameNumber')

# check by accessing a pokemon by name and number, get its weight: 
print("Weight of Bulbasaur is: ", pokemon_df_name.loc['bulbasaur 1']['Weight']) # should print 69

Weight of Bulbasaur is:  69


In [66]:
# function to filter list of pokemon using a string, which matches on pokemon name and number:
def filter_pokemon(pokemon_df, filter_string):
    return pokemon_df[pokemon_df['NameNumber'].str.contains(filter_string, case=False)]



In [67]:
# call function to filter pokemon by name and number: 
filtered_pokemon = filter_pokemon(pokemon_df, 'bulbasaur 1')
print(filtered_pokemon)

filter_pokemon2 = filter_pokemon(pokemon_df, 'ivysaur 2')	
print(filter_pokemon2)

filter_pokemon80 = filter_pokemon(pokemon_df, 'slowbro 80')
print(filter_pokemon80)

   Number       Name  Weight  Height  Base Experience   Ability   NameNumber
0       1  bulbasaur      69       7               64  overgrow  bulbasaur 1
   Number     Name  Weight  Height  Base Experience   Ability NameNumber
1       2  ivysaur     130      10              142  overgrow  ivysaur 2
    Number     Name  Weight  Height  Base Experience    Ability  NameNumber
79      80  slowbro     785      16              172  oblivious  slowbro 80


In [ ]:
def fetch_pokemon_data():
    pokemon_name = entry.get().lower()
    if not pokemon_name:
        messagebox.showerror("Error", "Please enter a Pokemon name.")
        return
    url = f'https://pokeapi.co/api/v2/pokemon/{pokemon_name}'
    response = requests.get(url)
    
    if response.status_code == 404:
        messagebox.showerror("Error", f"Pokemon '{pokemon_name}' not found.")
        return
    elif response.status_code != 200:
        messagebox.showerror("Error", "An error occurred.")
        return
    elif response.status_code == 200:
        data = response.json()
        display_pokemon_data(data)

def display_pokemon_data(data):
   
    result_text.delete(1.0, tk.END)
    result_text.insert(tk.END, f"Name: {data['name'].capitalize()}\n")
    result_text.insert(tk.END, f"Number: {data['id']}\n")
    result_text.insert(tk.END, f"Weight: {data['weight']}\n")
    result_text.insert(tk.END, f"Height: {data['height']}\n")
    result_text.insert(tk.END, f"Base Experience: {data['base_experience']}\n")
    result_text.insert(tk.END, "Abilities:\n")
    for ability in data['abilities']:
        result_text.insert(tk.END, f"- {ability['ability']['name']}\n") 
    result_text.insert(tk.END, "Types: " + ", ".join([t['type']['name'] for t in data['types']]) + "\n")


# create a main application window: 
root = tk.Tk()
root.title("Pokedex")

# Create and place the widgets:
tk.Label(root, text="Enter Pokemon Name:").pack(pady=5) 
entry = tk.Entry(root)
entry.pack(pady=5)

tk.Button(root, text="Fetch Data", command=fetch_pokemon_data).pack(pady=5)
result_text = tk.Text(root, height=10, width=50)
result_text.pack(pady=5)

# Start the main event loop:
root.mainloop()